In [1]:
import pandas

import numpy
import pygsheets
import datetime

In [2]:
excel_writer = pandas.ExcelWriter('AfricaGasTracker-Pipelines-SummarySheets-'+str(datetime.date.today())+'.xlsx')#, engine='xlsxwriter')


import Pipelines_Current dataset

In [3]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
#spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')
spreadsheet = gc.open_by_key('1jP1GJbl-BjLbwb9BTblt0EQQr2hyMGNgdVL4I9_wQWQ') # January 2022 pipelines

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df()#start='A2')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df()#start='A2')

#gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
#oil_pipes = oil_pipes.drop('WKTFormat', axis=1)
pipes_df_orig = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#pipes_df_orig = gas_pipes.copy()

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
owners_df_orig = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df()#start='A2')

# remove empty cells for pipes, owners
pipes_df_orig = pipes_df_orig[pipes_df_orig['PipelineName']!='']
owners_df_orig = owners_df_orig[owners_df_orig['Wiki']!='']
owners_df_orig.set_index('ProjectID', inplace=True)

In [4]:
country_ratios_df.replace('--', numpy.nan, inplace=True)

owners_df_orig.replace('',numpy.nan,inplace=True)
owners_df_orig.replace('--',numpy.nan,inplace=True)

pipes_df_orig.replace('--',numpy.nan,inplace=True)

In [5]:
region_df_orig = spreadsheet.worksheet('title', 'Region dictionary').get_as_df()
region_df_africa = region_df_orig.loc[region_df_orig['AfricaGasTracker']=='Yes']
region_df_touse = region_df_africa.copy()

### create Africa-specific dataframes for region, country_ratios_df, owners_df

In [6]:
country_ratios_df_touse = country_ratios_df.loc[country_ratios_df['Country'].isin(
                                            region_df_touse['Country'].tolist())]

owners_df_touse = owners_df_orig.loc[
    ~owners_df_orig['Countries'].apply(
        lambda x: set(x.split(', ')).isdisjoint(set(region_df_touse['Country'].tolist()))
    )]

#owners_df_touse = owners_df_orig.loc[owners_df_orig['Countries'].str.contains(
#                                            '|'.join(region_df_touse['Country'].tolist()))]

pipes_df_touse = pipes_df_orig.loc[
    ~pipes_df_orig['Countries'].apply(
        lambda x: set(x.split(', ')).isdisjoint(set(region_df_touse['Country'].tolist()))
    )]

#pipes_df_orig.loc[pipes_df_orig['Countries'].str.contains(
#                                            '|'.join(region_df_touse['Country'].tolist()))]

### set up info to analyze Owners tab

In [7]:
# get a set list of all owners
owner_column_list = ['Owner1', 
                     'Owner2', 
                     'Owner3', 
                     'Owner4', 
                     'Owner5', 
                     'Owner6', 
                     'Owner7', 
                     'Owner8', 
                     'Owner9', 
                     'Owner10', 
                     'Owner11']

percent_column_list = ['Owner1%', 
                     'Owner2%', 
                     'Owner3%', 
                     'Owner4%', 
                     'Owner5%', 
                     'Owner6%', 
                     'Owner7%', 
                     'Owner8%', 
                     'Owner9%', 
                     'Owner10%', 
                     'Owner11%']

owner_id_list = ['ID1', 
                 'ID2', 
                 'ID3', 
                 'ID4', 
                 'ID5', 
                 'ID6', 
                 'ID7', 
                 'ID8', 
                 'ID9', 
                 'ID10', 
                 'ID11']

### sum MergedKmByCountry and MergedKmByRegion

In [8]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(set(region_df_touse.Country.to_list())) #sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(set(region_df_touse.Region.tolist())) #sorted(list(set(country_ratios_df_touse['Region'])))

In [9]:
region_df_touse

,Country,Region,PipelineBubbleRegion,EEZNamesIfDifferent,EuropeanUnion,EuroGasTracker,AfricaGasTracker,Notes
0,Algeria,Middle East and North Africa,Middle East,,,,Yes,
1,Angola,Sub-Saharan Africa,Africa,,,,Yes,
2,Benin,Sub-Saharan Africa,Africa,,,,Yes,
3,Botswana,Sub-Saharan Africa,,,,,Yes,
4,Burkina Faso,Sub-Saharan Africa,,,,,Yes,
5,Burundi,Sub-Saharan Africa,,Burundi,,,Yes,
6,Cabo Verde,Sub-Saharan Africa,,,,,Yes,Cape Verde
7,Cameroon,Sub-Saharan Africa,Africa,,,,Yes,
8,Central African Republic,Sub-Saharan Africa,Africa,,,,Yes,
9,Chad,Sub-Saharan Africa,Africa,,,,Yes,


In [10]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']

In [11]:
fuel_type = 'Gas'
#fuel_type = 'Oil'
#fuel_type = 'NGL'

country_ratios_df_subset = country_ratios_df_touse.copy()[country_ratios_df_touse['Fuel']==fuel_type]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby('Region')['MergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['In Development (Proposed + Construction)'] = km_by_region[['Proposed','Construction']].sum(axis=1)
km_by_country['In Development (Proposed + Construction)'] = km_by_country[['Proposed','Construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.name = 'Region'
km_by_country.index.name = 'Country'

totals_row = km_by_region.sum(axis=0)
totals_row.name = 'Total'
km_by_region = km_by_region.append(totals_row)

totals_row = km_by_country.sum(axis=0)
totals_row.name = 'Total'
km_by_country = km_by_country.append(totals_row)

km_by_region.to_excel(excel_writer, 'Kilometers by region')
km_by_country.to_excel(excel_writer, 'Kilometers by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_33018/907386446.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_region = km_by_region.append(totals_row)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_33018/907386446.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_country = km_by_country.append(totals_row)


In [12]:
km_by_country

,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Country,,,,,,,,,
Algeria,1582.21,0.00,1582.21,190.30,0.00,9973.56,0.00,0.0,0.0
Angola,0.00,0.00,0.00,0.00,0.00,384.68,0.00,0.0,0.0
Benin,0.00,172.43,172.43,0.00,0.00,119.54,0.00,0.0,0.0
Botswana,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
Burkina Faso,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
Burundi,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
Cabo Verde,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
Cameroon,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
Central African Republic,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0


### create an excel file with all unique owners, not specific to fuel type

In [13]:
owner_list = []
for column in owner_column_list:
    owner_list+=list(owners_df_orig[column])
owner_list = list(set(owner_list))
# remove empty (no owner)
owner_list.remove(numpy.nan)
unique_owner_list = sorted(owner_list)

#pandas.Series(unique_owner_list).to_excel('AllFuels'+'UniqueOwnersList.xlsx')

## pipeline km by parent company (owner) and project status

In [14]:
fuel_type = 'Gas'
#fuel_type = 'Oil'
#fuel_type = 'NGL'

owners_df_subset = owners_df_touse.loc[owners_df_touse['Fuel']==fuel_type]

##################################################

owner_list_subset = []
for column in owner_column_list:
    owner_list_subset+=list(owners_df_subset[column])
owner_list_subset = list(set(owner_list_subset))
# remove empty (no owner)
owner_list_subset.remove(numpy.nan)
unique_owner_list_subset = sorted(owner_list_subset)

##################################################
# convert percents to fractions
##################################################
owners_df_fractions = owners_df_subset.copy()

for col in percent_column_list:
    print(col)
    owners_df_fractions[col] = owners_df_fractions[col].apply(lambda x: numpy.nan if x in [numpy.nan] 
                                          else x[:-1]).astype(float)/100

##################################################
# create km count by owner, status
##################################################
no_owner_info_count = 0
owners_km_by_status_df = pandas.DataFrame(0, index=unique_owner_list_subset, columns=status_list, dtype=object)

for status in status_list:
    
    owners_df_temporary = owners_df_fractions.loc[owners_df_fractions['Status']==status]
    
    for idx,row in owners_df_temporary.iterrows():
        
        ### how many owners are there?
        row_owners = list(row[owner_column_list])
        row_owners = [i for i in row_owners if str(i)!='nan']
        n_owners = row_owners.__len__()
        row_fractions = list(row[percent_column_list])
        row_fractions = [i for i in row_fractions if str(i)!='nan']

        ### if there are no owners listed, continue to next loop iteration
        if n_owners==0:
            no_owner_info_count+=1
            continue # doesn't complete the rest of the ifs

        ### now if row fractions is an empty list, create equal fractions list instead
        if row_fractions==[]:
            row_fractions = [1/n_owners]*n_owners

        pipe_length = row['LengthMergedKm']
        owner_km_fractions = numpy.array(row_fractions)*pipe_length

        for owner_idx in range(n_owners):
            owners_km_by_status_df.loc[row_owners[owner_idx]][status]+=owner_km_fractions[owner_idx]

owners_km_by_status_df.index.name = 'Parent Company'
owners_km_by_status_df['In Development (Proposed + Construction)'] = owners_km_by_status_df[['Proposed','Construction']].sum(axis=1)
owners_km_by_status_df = owners_km_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_km_by_status_df = owners_km_by_status_df[excel_status_list]

totals_row = owners_km_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_km_by_status_df = owners_km_by_status_df.append(totals_row)

owners_km_by_status_df = owners_km_by_status_df.astype(int)
owners_km_by_status_df.to_excel(excel_writer, 'Kilometers by owner')

Owner1%
Owner2%
Owner3%
Owner4%
Owner5%
Owner6%
Owner7%
Owner8%
Owner9%
Owner10%
Owner11%


/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_33018/240906509.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  owners_km_by_status_df = owners_km_by_status_df.append(totals_row)


In [19]:
owners_km_by_status_df

,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Parent Company,,,,,,,,,
Agip,0,0,0,0,0,143,0,0,0
Apache Corporation,0,0,0,0,0,374,0,0,0
Atlas Petroleum International,0,0,0,0,0,14,0,0,0
Axxela,510,0,510,0,0,0,0,0,0
BASF,0,0,0,0,0,129,0,0,0
...,...,...,...,...,...,...,...,...,...
Tunisian Company of Electricity and Gas,0,0,0,0,0,294,0,0,0
Volta River Authority,0,0,0,0,0,110,0,0,0
Waha Oil Company,0,0,0,0,0,354,0,0,0


### pipeline km by start year, type

In [15]:
pipes_started_eu = pipes_df_touse.copy()
#pipes_started_eu['StartYearLatest'].replace(numpy.nan,'',inplace=True)

pipes_started_eu = pipes_started_eu[(pipes_started_eu['Status'].isin(['Operating'])) &
                              (pipes_started_eu['Fuel']=='Gas')]
pipes_started_eu_sum = pipes_started_eu.groupby('StartYearEarliest')['LengthMergedKm'].sum()

In [16]:
km_by_start_year = pandas.DataFrame(index=list(range(1980,2023)), columns=['Gas pipeline km'])
km_by_start_year.index.name = 'Start year'

km_by_start_year['Gas pipeline km'] = pipes_started_eu_sum
km_by_start_year.replace(numpy.nan,0,inplace=True)

totals_row = km_by_start_year.sum(axis=0)
totals_row.name = 'Total'
km_by_start_year = km_by_start_year.append(totals_row)

km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_33018/2282834177.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_start_year = km_by_start_year.append(totals_row)


## save excel file

In [17]:
excel_writer.save()

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_33018/2638382022.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
